In [ ]:
# ms-python.python added
import os
try:
    os.chdir(os.path.join(os.getcwd(), 'pydata-exercise'))
    print(os.getcwd())
except:
    pass

 ## Pandas 练习

In [11]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

 ## 测试datafrmae传入的变化
 传入的是调用的引用.
 注意 `df=pd.DataFrame(a)`, 这句之后df就指向新的数据结构了,和传入的
 df就没有关系了. 但是append操作必须返回才能起效果,不像drop(inplace=True).

In [ ]:
def testdfgo(df):
    a = [{'A': 1, 'B': 2}, {'A': 3, 'B': 4}]
    print(df)
    # a = {'A': [1, 2], 'B': [3, 4]}
    # df = pd.DataFrame(a)
    df.append(a, ignore_index=True)
    print(df)
    df.drop(index=0, inplace=True)
    print(df)


df = pd.DataFrame([[0, 0], [5, 6]], columns=['A', 'B'])
testdfgo(df)
print(df)

 ## dataframe apply函数

 ### 处理每行的数据后并新增列

In [ ]:
data = {
    'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],
    'year': [2000, 2001, 2002, 2001, 2002, 2003],
    'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]
}
frame = pd.DataFrame(data)

# for row in frame.itertuples():
#    print(row)


def new_value0(row):
    return (5, 5)


def new_value(x=0, y=0):
    x = x + 5
    y = y * 5
    return x


def new_value1(row):
    row['new_year'] = row['year'] + 5
    row['new_pop'] = row['pop'] * 5
    return row


def new_value2(row, year, pop):
    x = row[year] + 5
    y = row[pop] * 5
    return x, y

In [ ]:
frame['new_year'] = frame.apply(lambda row: row['year'] + 5, axis=1)
frame['new_year'] = frame.apply(lambda row: new_value(row['year']), axis=1)

In [ ]:
frame = frame.apply(new_value1, axis=1)
print(frame)

In [ ]:
frame['new_year'], frame['new_pop'] = zip(*frame[['year', 'pop']].apply(new_value0, axis=1))
print(frame)

In [ ]:
frame['new_year'], frame['new_pop'] = zip(*frame[['year', 'pop']].apply(new_value2, axis=1, args=('year', 'pop')))
frame['new_year'], frame['new_pop'] = zip(*frame[['year', 'pop']].apply(new_value2, axis=1, year='year', pop='pop'))
# print('result')
print(frame)

 ### 理解zip和unzip的使用

In [ ]:
x1 = '5,6,7'
y1 = '1,2,3'
x1 = map(int, x1.split(','))
y1 = map(int, y1.split(','))
la = list(zip(x1, y1))
lb = []

for e in la:
    lb.append((e[0] + 1, e[1] + 1))

print(lb)
lb = list(zip(*lb))
print(lb)

 ## append函数
 主要是异常情况的处理

In [12]:
data = {
    'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],
    'year': [2000, 2001, 2002, 2001, 2002, 2003],
    'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]
}
f_name = '../data/hgj_gong_jiao_xian_lu 4.xlsx'
frame = pd.DataFrame(data)
nan_frame = pd.DataFrame()

book = load_workbook(f_name)
writer = pd.ExcelWriter(f_name, engine='openpyxl')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

try:
    nan_frame = pd.read_excel(f_name, sheet_name='123')
except ValueError:
    print('无此数据')
except:
    print('其他异常')
nan_frame = nan_frame.append(frame, ignore_index=True)
print(nan_frame)

if ('123' in writer.sheets):
    print('有此sheet页')
else:
    print('无此sheet页')

其他异常
    state  year  pop
0    Ohio  2000  1.5
1    Ohio  2001  1.7
2    Ohio  2002  3.6
3  Nevada  2001  2.4
4  Nevada  2002  2.9
5  Nevada  2003  3.2
无此sheet页


 ## 合并数据-merge函数的理解和使用

### 初始化测试数据

In [1]:
product = pd.DataFrame({
    'id':
    np.arange(101, 111),
    'date':
    pd.date_range(start='20200505', periods=10),
    'money': [5, 4, 65, 10, 15, 20, 35, 16, 6, 20],
    'product': ['苏打水', '可乐', '牛肉干', '老干妈', '菠萝', '冰激凌', '洗面奶', '洋葱', '牙膏', '薯片'],
    'department': ['饮料', '饮料', '零食', '调味品', '水果', np.nan, '日用品', '蔬菜', '日用品', '零食'],
    'origin': ['China', ' China', 'America', 'China', 'Thailand', 'China', 'america', 'China', 'China', 'Japan']
})
product

,id,date,money,product,department,origin
0,101,2020-05-05,5,苏打水,饮料,China
1,102,2020-05-06,4,可乐,饮料,China
2,103,2020-05-07,65,牛肉干,零食,America
3,104,2020-05-08,10,老干妈,调味品,China
4,105,2020-05-09,15,菠萝,水果,Thailand
5,106,2020-05-10,20,冰激凌,NaN,China
6,107,2020-05-11,35,洗面奶,日用品,america
7,108,2020-05-12,16,洋葱,蔬菜,China
8,109,2020-05-13,6,牙膏,日用品,China
9,110,2020-05-14,20,薯片,零食,Japan


In [ ]:
product_1 = pd.DataFrame({
    'id': np.arange(110, 113),
    'date': pd.date_range(start='20200514', periods=3),
    'money': [20, 7, 14],
    'product': ['薯片', 'qiqudan', 'cho'],
    'department': ['零食', '零食', '零食'],
    'origin': ['China', 'China', 'China']
})
product_1

### 验证merge函数不同参数的用法

In [ ]:
product_new = pd.merge(product, product_1, on='id', how='inner')
product_new

In [ ]:
product_new = pd.merge(product, product_1, on='id', how='right')
product_new

#### 不指定特定的on参数
对全column进行比较,相当于intersection(inner)和unioin(outer)

In [ ]:
# 注意110的产地,一个是Japan,一个是China,merge的时候会当成不同数据
product_new = pd.merge(product, product_1, how='outer')
product_new
product_1.loc[product_1['id'] == 110, 'origin'] = 'Japan'
product_new = pd.merge(product, product_1, how='outer')
product_new

In [ ]:
# 另一种写法,效果同上.
product.merge(product_1, how='outer')